In [10]:
from __future__ import print_function
import argparse
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import MNIST
from torch.autograd import Variable

In [11]:
# settings
batch_size = 128
batch_size_test = 1000
n_epochs = 10
learning_rate = 0.01
momentum = 0.9
use_cuda = True
rnd_seed = 1
log_interval = 10

In [12]:
# CapsNet
from capsnet import CapsNetWithReconstruction, CapsNet, ReconstructionNet

In [14]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Pad(2), transforms.RandomCrop(28),
                           transforms.ToTensor()
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)

In [50]:
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
    out = conv1(data)
    out1 = F.max_pool2d(out,2)
    print('Input:\t\t\t', data.shape)
    print('After Conv2d:\t\t', out.shape)
    print('After MaxPool2d:\t', out1.shape)
    #if batch_idx>0:
    break

Input:			 torch.Size([128, 1, 28, 28])
After Conv2d:		 torch.Size([128, 10, 24, 24])
After MaxPool2d:	 torch.Size([128, 10, 12, 12])


In [56]:
class OneLayerConv2d(nn.Module):
    def __init__(self):
        super(OneLayerConv2d, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=10, 
                               kernel_size=5, bias=True)
        
    def forward(self, x):
        return self.conv(x)

In [57]:
net = OneLayerConv2d()
print(net)

OneLayerConv2d(
  (conv1): Conv2d (2, 10, kernel_size=(5, 5), stride=(1, 1))
)


In [76]:
params = list(net.parameters())
print(len(params), "-> parameters for weights and biases")
print(params[0].size())
print("Number of trainable parameters:\t",
     reduce(lambda x, y: x*y, params[0].size()))

2 -> parameters for weights and biases
torch.Size([10, 2, 5, 5])
Number of trainable parameters:	 500


In [63]:
print(list(net.parameters())[0].size())
print(list(net.parameters())[0])

torch.Size([10, 2, 5, 5])
Parameter containing:
(0 ,0 ,.,.) = 
  0.1066  0.0026 -0.0060  0.1339  0.0063
 -0.0425 -0.1411  0.0214 -0.0909 -0.0752
 -0.0764 -0.0703  0.0554  0.0164 -0.1044
  0.1413 -0.1279  0.0555  0.0241  0.0328
  0.1375  0.0222 -0.0564 -0.0859 -0.0329

(0 ,1 ,.,.) = 
  0.1097  0.1307 -0.1246 -0.0089 -0.0799
  0.0836  0.1281  0.0551  0.0797  0.1144
 -0.1012 -0.0591  0.0969 -0.0979  0.0029
 -0.1254 -0.0456 -0.0388 -0.0642  0.0758
 -0.0726  0.0712  0.0626 -0.1267 -0.0501

(1 ,0 ,.,.) = 
  0.1028 -0.0568 -0.1045 -0.0854 -0.0940
 -0.1160 -0.0787  0.0583  0.1102  0.1147
 -0.0188  0.0131 -0.0100  0.1199  0.0963
 -0.0911 -0.0688  0.1021  0.0111 -0.0063
 -0.0598 -0.0090  0.0249 -0.0780  0.0704

(1 ,1 ,.,.) = 
 -0.0007 -0.0162  0.0171 -0.0254 -0.1341
  0.0210  0.1134  0.0825  0.1352  0.0877
 -0.1370 -0.0725 -0.0253 -0.1200  0.0305
 -0.0827  0.0857 -0.0488  0.1386  0.0436
 -0.0306  0.0224 -0.0571  0.0501  0.1226

(2 ,0 ,.,.) = 
 -0.0013 -0.0700  0.0545  0.1234  0.0104
  0.1113 -0.